In [5]:
import os
import openai
import sys

sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


openai.api_key  = os.environ['OPENAI_API_KEY']

In [6]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("text.pdf")
pages = loader.load()
page = pages[0]
print(page.page_content)


  GOSHEN  OLATUNBOSUN  
ICAN  PROFESSIONAL  STAGE  
  
  
   Ikorodu, Lagos •• Cell: +2348119592939  • goshenbosun002@gmail.com    
Professional Summary  
An accounting graduate seeking an opportunity in accounting and finance to utilize my skills in your industry, in 
addition, I’m highly motivated, detailed oriented, and knowledgeable in the use of the most common accounting 
software programs. Strong work ethic also commitment to integrity and accurate record keeping. Eager to join a team 
and help improve an organizati on’s financial focus.  
Skills  
● Solid working knowledge of MS office with a strong 
level of proficiency in Excel  
● Excellent organizational skills with demonstrated 
ability to effectively prioritize to meet strict deadlines.  
● Exceptional knowledge of accounting concepts as 
well as mathematics, accounting and finance topics.  ● Preparation of accurate financial accounts and reports 
to comply with accounting principles and practices.  
● Proven ability to q

In [7]:
page2 = pages[1]
print(page2.page_content)


Zioneeel International College  
Junior Secondary Certificate Examination        2013  
Royal Diadem Demonstration School         2009  
 
Certifications  
Institute of Chartered Accountant of Nigeria (Professional Stage)  
Digital Literacy (Microsoft)  
Certified in digital marketing (Google)  
Digital marketing booth camp (Digic lan Africa)  
Advanced Social Media Management (Simplilearn)  
 
 
REFEREES 0 
Ilemobayo  A. Akingbomdere   [FCA]  
CEO  
IleAkin Financial Consulting  
+23423181112  
 
Mr. Olatunbosun Olaniyi  
Director  
Mutual Alliance, Lagos State  
+2348062147426  
 
 
 
 


In [8]:
print(page2.metadata)

{'source': 'text.pdf', 'page': 1}


In [9]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

text_splitter_1 = CharacterTextSplitter( separator= "", chunk_size = 150, chunk_overlap = 10 , length_function = len )

docs = text_splitter_1.split_documents(pages)


print(len(docs))
print(len(pages))

25
2


#Convert the text/splits into embeddings


In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [11]:


#embedding1 = embeddings.embed_query("i like dogs")
#embedding2 = embeddings.embed_query("i like canines")
#enbedding3 = embeddings.embed_query("the weather is ugly outside")

In [12]:
import numpy as np

#print(np.dot(embedding1 , embedding2))
#print(np.dot(embedding2, enbedding3))

Saving our vector embeddings into chromadb. ##chroma is light weight and in memory##

In [13]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma/'

#!rm -rf ./docs/chroma 

In [14]:
#Actually creating the chromadb vector database.
vectordb = Chroma.from_documents(
    documents= docs,
    embedding= embeddings,
    persist_directory= persist_directory
)

print(vectordb._collection.count())

50


In [15]:
question = "does he have good speaking and penning with online expertise?"

response = vectordb.similarity_search(question, k=3)

print(len(response))

3


In [16]:
print(response[0].page_content)
print(response[1].page_content)
print(response[2].page_content)

Proven ability to quickly learn and use new 
technologies  
● Excellent written and communication skills.  
● Excellent understanding of Digital mark
Proven ability to quickly learn and use new 
technologies  
● Excellent written and communication skills.  
● Excellent understanding of Digital mark
● Solid working knowledge of MS office with a strong 
level of proficiency in Excel  
● Excellent organizational skills with demonstrated 
ability t


In [17]:
vectordb.persist()

Retrieval Using maximum marginal Relevanve

In [18]:
question2 = "does he have good speaking and penning with online expertise?"

response2 = vectordb.max_marginal_relevance_search(question2, k=3)

print(response2[0].page_content)
print("~~~")
print(response2[1].page_content)
print("~~~")
print(response2[2].page_content)

Proven ability to quickly learn and use new 
technologies  
● Excellent written and communication skills.  
● Excellent understanding of Digital mark
~~~
GOSHEN  OLATUNBOSUN  
ICAN  PROFESSIONAL  STAGE  
  
  
   Ikorodu, Lagos •• Cell: +2348119592939  • goshenbosun002@gmail.com    
Professional Summa
~~~
student learning and performances and communicated necessary information to students  
● I helped secure a Facebook grant of half a million naira(#500


Retrieval using SelfQuery (i.e LLM aided retrieval)

In [19]:
#appropriate imports 

from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [32]:
metadata_field_info = [
    AttributeInfo(
        name = "source",
        description = "The origin on the chunk. should be 'text.pdf' ",
        type = "string",
    ),
    AttributeInfo(
        name = "page",
        description = "The page from which the chunk was taken. There are two pages in the document",
        type = "integer",
    ),
]

document_content_description = "Goshen's Resume (CV)"
llm = OpenAI(temperature= 0)

retriever = SelfQueryRetriever.from_llm(
    llm, 
    vectordb,
    document_content_description, 
    metadata_field_info, 
    verbose=True
)

ImportError: Cannot import lark, please install it with 'pip install lark'.